# Quantum kicked rotor (Floquet) via split-operator + FFT

In [2]:
#!/usr/bin/env python3
# part2.py — Quantum kicked rotor (Floquet) via split-operator + FFT
#
# Model (dimensionless, period = 1):
#   H = p^2/2 + K cos(theta) * sum_n delta(t-n)
# Floquet step (symmetric splitting):
#   U ≈ exp(-i p^2/(4hbar)) exp(-i K cosθ / hbar) exp(-i p^2/(4hbar))
#
# Outputs:
#   out_quantum/kr_m2_vs_n.csv      : <m^2>, <p^2>, norm vs n
#   out_quantum/kr_final_pm.csv     : momentum distribution |psi_m|^2 at final step
#   out_quantum/kr_final_ptheta.csv : angle distribution |psi(theta)|^2 at final step

from __future__ import annotations

import os
import numpy as np
import matplotlib.pyplot as plt

OUTDIR = "out_quantum"
FIGDIR = os.path.join(OUTDIR, "figs")
os.makedirs(FIGDIR, exist_ok=True)


In [3]:
def fft_theta_to_m(psi_theta: np.ndarray) -> np.ndarray:
    """
    Transform a wavefunction from angle representation to momentum representation.

    This uses a unitary FFT convention:
    - Input:  psi_theta[j] = psi(θ_j) sampled on a uniform grid θ_j in [0, 2π).
    - Output: psi_m[k]     = ψ_m on an integer angular-momentum grid m centered at 0.

    The returned array is fftshift'ed so that m=0 is at the center of the array.
    """
    n = psi_theta.size
    return np.fft.fftshift(np.fft.fft(psi_theta)) / np.sqrt(n)


def ifft_m_to_theta(psi_m: np.ndarray) -> np.ndarray:
    """
    Transform a wavefunction from momentum representation back to angle representation.

    This uses the inverse of fft_theta_to_m with the same unitary convention:
    - Input:  psi_m[k]     = ψ_m on centered integer grid m (fftshift'ed).
    - Output: psi_theta[j] = psi(θ_j) on θ-grid in [0, 2π).

    The returned array is normalized consistently with the forward transform.
    """
    n = psi_m.size
    return np.fft.ifft(np.fft.ifftshift(psi_m)) * np.sqrt(n)


def make_grids(n: int):
    """
    Build the canonical grids for the kicked rotor on a finite Hilbert space.

    Parameters
    ----------
    n : int
        Grid size (Hilbert space dimension). Typically a power of 2 for fast FFT.

    Returns
    -------
    theta : ndarray (n,)
        Uniform angle grid in [0, 2π).
    m : ndarray (n,)
        Integer angular momentum grid centered at 0: m = -n/2, ..., n/2-1.
    """
    theta = 2.0 * np.pi * np.arange(n) / n
    m = np.arange(-n // 2, n // 2, dtype=np.int64)
    return theta, m


def normalize(psi: np.ndarray) -> np.ndarray:
    """
    Normalize a complex state vector to unit norm.

    Parameters
    ----------
    psi : ndarray (complex)
        State vector in any representation.

    Returns
    -------
    psi_unit : ndarray
        psi / ||psi||, with ||psi|| = sqrt(<psi|psi>).

    Raises
    ------
    ValueError
        If the norm is zero (numerically or exactly).
    """
    norm = np.sqrt(np.vdot(psi, psi).real)
    if norm == 0:
        raise ValueError("Psi has zero norm.")
    return psi / norm


def kicked_rotor_step(
    psi_theta: np.ndarray,
    theta: np.ndarray,
    m: np.ndarray,
    K: float,
    hbar_eff: float,
) -> np.ndarray:
    """
    Apply one Floquet period of the quantum kicked rotor using symmetric splitting.

    Model (dimensionless, period = 1):
        H = p^2/2 + K cos(theta) * sum_n δ(t-n)

    Floquet step (symmetric split-operator):
        U ~ exp[-i p^2/(4hbar)] * exp[-i K cos(theta)/hbar] * exp[-i p^2/(4hbar)]

    Implementation details
    ----------------------
    - Kick is diagonal in theta-space.
    - Free evolution is diagonal in momentum-space (m basis).
    - FFT/iFFT swaps between theta and m bases.
    """
    # In this discretization p = m * hbar_eff, therefore:
    # exp[-i p^2/(2hbar_eff)] = exp[-i hbar_eff * m^2 / 2]
    # half-step: exp[-i hbar_eff * m^2 / 4]
    half_kin = np.exp(-1j * (hbar_eff * (m.astype(np.float64) ** 2)) / 4.0)
    kick = np.exp(-1j * (K * np.cos(theta)) / hbar_eff)

    psi_m = fft_theta_to_m(psi_theta)
    psi_m *= half_kin
    psi_theta = ifft_m_to_theta(psi_m)

    psi_theta *= kick

    psi_m = fft_theta_to_m(psi_theta)
    psi_m *= half_kin
    psi_theta = ifft_m_to_theta(psi_m)

    return psi_theta


def run_simulation(
    n_grid: int = 2048,
    n_steps: int = 4000,
    K: float = 5.0,
    hbar_eff: float = 1.0,
    sigma_m0: float = 2.0,
    m0: float = 0.0,
    seed: int = 1234,
    outdir: str = "out_quantum",
    snap_stride: int = 10,          # <-- NEW
    save_snapshots: bool = True,    # <-- NEW
):
    """
    Run a kicked-rotor simulation and write CSV outputs for post-processing/plots.

    Initial state
    -------------
    A Gaussian wavepacket in momentum (m) centered at m0 with width sigma_m0,
    then transformed to theta representation.

    Outputs (written to outdir)
    ---------------------------
    - kr_m2_vs_n.csv        : n, <m^2>, <p^2>, norm
    - kr_final_pm.csv       : m, |psi_m|^2 at final step
    - kr_final_ptheta.csv   : theta, |psi(theta)|^2 at final step
    - kr_pm_snapshots.npz   : n_snap, m, pm(T,M) for heatmap (if save_snapshots)
    """
    rng = np.random.default_rng(seed)
    os.makedirs(outdir, exist_ok=True)

    theta, m = make_grids(n_grid)

    # Initial Gaussian in m
    psi_m0 = np.exp(-0.5 * ((m - m0) / sigma_m0) ** 2).astype(np.complex128)
    psi_m0 *= np.exp(1j * 2.0 * np.pi * rng.random())  # global phase
    psi_m0 = normalize(psi_m0)

    psi_theta = ifft_m_to_theta(psi_m0)
    psi_theta = normalize(psi_theta)

    # --- NEW: snapshot buffers for heatmap ---
    pm_snaps = []
    n_snaps = []

    csv_path = os.path.join(outdir, "kr_m2_vs_n.csv")
    with open(csv_path, "w", encoding="utf-8") as f:
        f.write("n,m2,p2,norm\n")
        for n in range(n_steps + 1):
            psi_m = fft_theta_to_m(psi_theta)
            pm = (np.abs(psi_m) ** 2).real
            pm /= pm.sum()

            # --- NEW: store snapshots every snap_stride (and always last) ---
            if save_snapshots and (n % snap_stride == 0 or n == n_steps):
                n_snaps.append(n)
                pm_snaps.append(pm.astype(np.float32))

            m2 = float((pm * (m.astype(np.float64) ** 2)).sum())
            p2 = float((hbar_eff**2) * m2)
            norm = float(np.vdot(psi_theta, psi_theta).real)

            f.write(f"{n},{m2:.12e},{p2:.12e},{norm:.12e}\n")

            if n == n_steps:
                break

            psi_theta = kicked_rotor_step(psi_theta, theta, m, K=K, hbar_eff=hbar_eff)
            psi_theta = normalize(psi_theta)

    # --- NEW: write snapshots file for heatmap panel ---
    if save_snapshots and len(pm_snaps) > 0:
        pm_arr = np.vstack(pm_snaps)  # shape (T, M)
        np.savez(
            os.path.join(outdir, "kr_pm_snapshots.npz"),
            n_snap=np.array(n_snaps, dtype=np.int64),
            m=m.astype(np.int64),
            pm=pm_arr,
        )

    # Final distributions
    psi_m = fft_theta_to_m(psi_theta)
    pm = (np.abs(psi_m) ** 2).real
    pm /= pm.sum()

    ptheta = (np.abs(psi_theta) ** 2).real
    ptheta /= ptheta.sum()

    np.savetxt(
        os.path.join(outdir, "kr_final_pm.csv"),
        np.column_stack([m, pm]),
        delimiter=",",
        header="m,prob",
        comments="",
    )
    np.savetxt(
        os.path.join(outdir, "kr_final_ptheta.csv"),
        np.column_stack([theta, ptheta]),
        delimiter=",",
        header="theta,prob",
        comments="",
    )

    return csv_path


In [4]:
if __name__ == "__main__":
    # Suggested poster-friendly defaults:
    # - Try two regimes in separate runs:
    #   (A) K small-ish: more regular
    #   (B) K large: classically diffusive, quantum shows saturation (dynamical localization)
    path = run_simulation(
        n_grid=2048,
        n_steps=4000,
        K=5.0,
        hbar_eff=1.0,
        sigma_m0=2.0,
        m0=0.0,
        seed=1234,
        outdir="out_quantum",
    )
    print("Wrote:", path)


Wrote: out_quantum/kr_m2_vs_n.csv
